In [1]:
from urllib.request import urlopen
import json
import time

In [2]:
response = urlopen('https://api.openf1.org/v1/sessions?year>=2022')
data = json.loads(response.read().decode('utf-8'))

In [3]:
sessions=[]
for session in data:
    if session["session_type"] == "Race":
        sessions.append({
            "session_key": session["session_key"],
            "meeting_key": session["meeting_key"],
            "circuit_key": session["circuit_key"],
            "country_code": session["country_code"],
            #"country_key": session["country_key"],
            "year": session["year"],
            "session_name": session["session_name"],
            "session_type":session["session_type"],
        })
len(sessions)

64

In [ ]:
import csv
import os 

for session in sessions:
    time.sleep(5)
    session_key = session["session_key"]
    circuit_key = session["circuit_key"]
    country_code = session["country_code"]
    session_name = session["session_name"]
    year = session["year"]

    print(f"Starting {session_key} {circuit_key} {country_code} {session_name} {year}")

    response = urlopen(f'https://api.openf1.org/v1/drivers?session_key={session_key}')
    drivers = json.loads(response.read().decode('utf-8'))
    
    unique_drivers = {}
    for driver in drivers:
        unique_drivers[driver["driver_number"]]={
            "driver_name": driver["full_name"],
            "team": driver["team_name"]
        }

    response = urlopen(f'https://api.openf1.org/v1/position?session_key={session_key}')
    positions = json.loads(response.read().decode('utf-8'))

    for driver_num in list(unique_drivers.keys()):
        cur_driver_positions = []
        for pos in positions:
            if pos["driver_number"] == driver_num:
                cur_driver_positions.append(pos)
        unique_drivers[driver_num].update({"finishing_pos":cur_driver_positions[-1]["position"]})
    
    try:
        file_exists = os.path.isfile("data.csv")
        with open("data.csv",mode="a", newline='') as f:
            writer = csv.writer(f)
            
            if not file_exists:
                writer.writerow(["driver_id","driver_name","team","session_name","circuit_key","country_code","year","finishing_pos"])

            for driver_id, info in unique_drivers.items():
                writer.writerow([
                    driver_id,
                    info["driver_name"],
                    info["team"],
                    session_name,
                    circuit_key,
                    country_code,
                    year,
                    info["finishing_pos"]
                ])

    except Exception as e:
        print(e)
        

Starting 7953 63 BRN Race 2023
Starting 7779 149 KSA Race 2023
Starting 7787 10 AUS Race 2023
Starting 9069 144 AZE Sprint 2023
Starting 9070 144 AZE Race 2023
Starting 9078 151 USA Race 2023
Starting 9094 22 MON Race 2023
Starting 9102 15 ESP Race 2023
Starting 9110 23 CAN Race 2023
Starting 9117 19 AUT Sprint 2023
Starting 9118 19 AUT Race 2023
Starting 9126 2 GBR Race 2023
Starting 9133 4 HUN Race 2023
Starting 9140 7 BEL Sprint 2023


HTTPError: HTTP Error 500: Internal Server Error

In [ ]:
response = urlopen(f'https://api.openf1.org/v1/drivers?session_key={session_key}')
drivers = json.loads(response.read().decode('utf-8'))